In [80]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import re
from src.data_items import add_suffixes
from selenium.webdriver.common.keys import Keys
import pickle

In [81]:
pricedf = pd.read_csv('src/scraping/atx_listings_trulia.csv')

In [82]:
geodf = pd.read_csv('data/Austin_addresses.csv')

/Users/austinmadert/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
geodf['c_zip_code'] = geodf['c_zip_code'].astype(str).map(lambda x: x.replace('.0', ''))
geodf.head()

,address,address1,address2,c_street_number,c_street,c_unit,c_city,c_state,c_zip_code,g_street_number,g_street,g_unit,g_city,g_state,g_zip_code,g_county,lat,lon
0,20813 PENNY ROYAL DR PFLUGERVILLE TX 78660,20813 PENNY ROYAL DR PFLUGERVILLE TX 78660,PENNY ROYAL DR PFLUGERVILLE TX 78660,20813,PENNY ROYAL DRIVE,NaN,PFLUGERVILLE,TX,78660,20813,Penny Royal Drive,NaN,Pflugerville,Texas,78660.0,Travis County,30.484112,-97.563114
1,10604 FOUNDATION ROAD AUS1 TN TX 78726,10604 FOUNDATION RD AUS1 TN TX 78726,FOUNDATION RD AUS1 TN TX 78726,10604,FOUNDATION ROAD,NaN,AUSTIN,TX,78726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12200 PRATOLINA DR AUSTIN TX 78739,12200 PRATOLINA DR AUSTIN TX 78739,PRATOLINA DR AUSTIN TX 78739,12200,PRATOLINA DRIVE,NaN,AUSTIN,TX,78739,12200,Pratolina Drive,NaN,Austin,Texas,78739.0,Travis County,30.186914,-97.928031
3,311 EVENING GROSBEAK DRIVE PFLUGERVILLE TX 78660,311 EVENING GROSBEAK DR PFLUGERVILLE TX 78660,EVENING GROSBEAK DR PFLUGERVILLE TX 78660,311,EVENING GROSBEAK DRIVE,NaN,PFLUGERVILLE,TX,78660,311,Evening Grosbeak Drive,NaN,Pflugerville,Texas,78660.0,Travis County,30.446861,-97.627389
4,8201 EDGEMOOR PLACE AUSTIN TX 78749,NaN,NaN,8201,EDGEMOOR PLACE,NaN,AUSTIN,TX,78749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
pricedf.head()

,address,bathrooms,bedrooms,city_state_zip,house_type,price,sqft,url
0,4159 Steck Ave #102,2 Bathrooms,3 Bedrooms,"Austin, TX 78759",Condo,"$280,000","1,427 Square Feet",https://www.trulia.com/p/tx/austin/4159-steck-...
1,15810 De Peer Cv,3 Bathrooms,4 Bedrooms,"Austin, TX 78717",Single-Family Home,"$299,000","2,224 Square Feet",https://www.trulia.com/p/tx/austin/15810-de-pe...
2,8201 Lazy Ln,1 Bathroom,3 Bedrooms,"Austin, TX 78757",Single-Family Home,"$329,900","1,005 Square Feet",https://www.trulia.com/p/tx/austin/8201-lazy-l...
3,14511 Fitzgibbon Dr,2 Bathrooms,3 Bedrooms,"Austin, TX 78725",Single-Family Home,"$229,874","1,499 Square Feet",https://www.trulia.com/p/tx/austin/14511-fitzg...
4,10304 Bankhead Dr,2 Bathrooms,3 Bedrooms,"Austin, TX 78757",Single-Family Home,"$237,501","1,295 Square Feet",https://www.trulia.com/p/tx/austin/10304-bankh...


In [85]:
pricedf['price'] = pricedf['price'].map(lambda x: str(x).replace(',', '').replace('$',''))

In [86]:
pricedf['sqft'] = pricedf['sqft'].map(lambda x: str(x).replace(',', '').replace(' Square Feet',''))

In [87]:
pricedf['sqft'].fillna(0, inplace=True)

In [88]:
pricedf['sqft'] = pricedf['sqft'].map(lambda x: 0 if x == 'nan' else x)

In [89]:
pricedf.house_type.unique()

array(['Condo', 'Single-Family Home', 'Mobile/Manufactured', 'Townhouse',
       nan, 'Lot/Land', '4,267 Square Feet', 'Lot Size: 9060 sqft',
       'Lot Size: 9496 sqft', '456 Square Feet', 'Lot Size: 8494 sqft',
       'Lot Size: 10541 sqft', 'Lot Size: 1.04 acres',
       'Lot Size: 0.61 acres', 'Lot Size: 1.00 acres', '952 Square Feet',
       'Lot Size: 1.53 acres', 'Lot Size: 0.29 acres',
       '3,565 Square Feet', 'Lot Size: 8973 sqft', '4,089 Square Feet',
       'Lot Size: 9583 sqft', 'Lot Size: 8145 sqft',
       'Lot Size: 9278 sqft', '1,130 Square Feet', 'Lot Size: 0.66 acres',
       '1,302 Square Feet', '2,100 Square Feet', 'Lot Size: 8668 sqft',
       'Lot Size: 7431 sqft', '4,000 Square Feet', 'Lot Size: 3571 sqft',
       '4,788 Square Feet', '2,201 Square Feet', 'Lot Size: 10.48 acres',
       'Lot Size: 7100 sqft', 'Multi-Family', 'house_type',
       'Lot Size: 11.40 acres', 'Lot Size: 0.28 acres',
       '1,700 Square Feet', '3,504 Square Feet', 'Lot Size: 1.62 a

In [90]:
pricedf[pricedf['house_type'] == 'Lot Size: 9496 sqft']

,address,bathrooms,bedrooms,city_state_zip,house_type,price,sqft,url
135,403 Oertli Ln,"1,568 Square Feet",Multi-Family,"Austin, TX 78753",Lot Size: 9496 sqft,260000,2 Bathrooms,https://www.trulia.com/p/tx/austin/403-oertli-...


In [91]:
 def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [92]:
newpricedf = pricedf[pricedf['sqft'].map(lambda x: True if str(x).isdigit() else False)].copy()

In [93]:
newpricedf['price'] = newpricedf['price'].astype(int)
newpricedf['sqft'] = newpricedf['sqft'].astype(int)

In [94]:
newpricedf["price_per_sqft"] = newpricedf['price'] / newpricedf['sqft']

In [95]:
newpricedf.drop_duplicates(inplace=True)

In [96]:
newpricedf.shape

(662, 9)

In [97]:
newpricedf.address.is_unique

False

In [98]:
address = newpricedf.address
newpricedf[address.isin(address[address.duplicated()])]

,address,bathrooms,bedrooms,city_state_zip,house_type,price,sqft,url,price_per_sqft
221,14921 Custis Ln,2 Bathrooms,4 Bedrooms,"Austin, TX 78725",Single-Family Home,241990,2040,https://www.trulia.com/property/4061230422-149...,118.622549
569,14921 Custis Ln,NaN,Unknown,"Austin, TX 78725",NaN,241990,0,https://www.trulia.com/p/tx/austin/14921-custi...,inf


In [99]:
newpricedf.drop(569, inplace=True)

In [100]:
newpricedf.address.is_unique

True

In [101]:
newpricedf.address = newpricedf.address.str.upper()

In [102]:
addlist = newpricedf.address.map(lambda x: x.split(' '))

In [103]:
addset = set(add_suffixes)

In [104]:
def cleanup(string):
    string = string.upper().strip('.').strip(',')
    if string in addset:
        string = add_suffixes[string]
    return string

In [105]:
addclean = [ [ cleanup(word) for word in address] for address in addlist ]

In [106]:
def findPound(address):
    result = ''
    for word in address:
        if re.search('#', word):
            result = word
        continue
    if result == '':
        result = 'Nan'
    return result

In [107]:
apts = [findPound(address) for address in addclean]

In [108]:
addclean = [[word for word in address if not re.search('#', word)] for address in addclean]

In [109]:
joinedclean = [' '.join(address) for address in addclean]

In [110]:
joinedclean

['4159 STECK AVENUE',
 '15810 DE PEER COVE',
 '8201 LAZY LANE',
 '14511 FITZGIBBON DRIVE',
 '10304 BANKHEAD DRIVE',
 '8505 GRAYLEDGE DRIVE',
 '1800 LAVACA STREET',
 '1718 SUMMIT VIEW',
 '11825 SHROPSHIRE BOULEVARD',
 '10200 COPPER RIDGE COVE',
 '8207 DIXON DRIVE',
 '541 CATALINA LANE',
 '166 BRIARPATCH COURT',
 '1401 E RUNDBERG LANE',
 '2020 DRY SEASON TRAIL',
 '2415 MELISSA OAKS LANE',
 '1101 E PARMER LANE',
 '10208 COLONIAL CLUB DRIVE',
 '11809 PALISADES PARKWAYS',
 '2018 W RUNDBERG LANE',
 '8913 WILEY WAY',
 '5007 EDENBOURGH LANE',
 '10503 GOLDEN QUAIL DRIVE',
 '15103 NUTTALL DRIVE',
 '3511 WESTLAKE DRIVE',
 '2300 PORTOFINO RIDGE DRIVE',
 '9012 TALL SKY TRACE',
 '2203 E 51ST STREET',
 '4104 SHIMMERING COVE',
 '10104 JUPITER HILLS DRIVE',
 '14217 HANDEL DRIVE',
 '1310 S 1ST STREET',
 '7712 HIDDEN BROOK COURT',
 '3008 FOXTON COVE',
 '2600 SCENIC DRIVE',
 '113 EIGLEHART ROAD',
 '3840 FAR WEST BOULEVARD',
 '2617 NATIONAL PARKS BOULEVARD',
 '15013 ROODY LANE',
 '15529 STAKED PLAINS LOOP'

In [111]:
newpricedf['city_state_zip'] = newpricedf['city_state_zip'].map(lambda x: cleanup(x).replace('AUSTIN,', ' AUSTIN'))

In [112]:
newpricedf['adj_address'] = joinedclean + newpricedf['city_state_zip']

In [113]:
newpricedf['apts'] = apts
newpricedf.head()

,address,bathrooms,bedrooms,city_state_zip,house_type,price,sqft,url,price_per_sqft,adj_address,apts
0,4159 STECK AVE #102,2 Bathrooms,3 Bedrooms,AUSTIN TX 78759,Condo,280000,1427,https://www.trulia.com/p/tx/austin/4159-steck-...,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102
1,15810 DE PEER CV,3 Bathrooms,4 Bedrooms,AUSTIN TX 78717,Single-Family Home,299000,2224,https://www.trulia.com/p/tx/austin/15810-de-pe...,134.442446,15810 DE PEER COVE AUSTIN TX 78717,Nan
2,8201 LAZY LN,1 Bathroom,3 Bedrooms,AUSTIN TX 78757,Single-Family Home,329900,1005,https://www.trulia.com/p/tx/austin/8201-lazy-l...,328.258706,8201 LAZY LANE AUSTIN TX 78757,Nan
3,14511 FITZGIBBON DR,2 Bathrooms,3 Bedrooms,AUSTIN TX 78725,Single-Family Home,229874,1499,https://www.trulia.com/p/tx/austin/14511-fitzg...,153.351568,14511 FITZGIBBON DRIVE AUSTIN TX 78725,Nan
4,10304 BANKHEAD DR,2 Bathrooms,3 Bedrooms,AUSTIN TX 78757,Single-Family Home,237501,1295,https://www.trulia.com/p/tx/austin/10304-bankh...,183.398456,10304 BANKHEAD DRIVE AUSTIN TX 78757,Nan


In [114]:
newpricedf.to_pickle('src/scraping/trulia/sel_scrape/trulscraped_df.pkl')

In [75]:
newpricedf.shape

(661, 7)

In [36]:
mergedf = newpricedf.merge(geodf, how='left', left_on='adj_address', right_on='address')

In [37]:
geodf['spaces'] = ' '

In [38]:
geodf['adj_address'] = geodf['c_street_number'] + geodf['spaces'] + geodf['c_street'] + geodf['spaces'] + geodf['c_city'] + geodf['spaces'] + geodf['c_state'] + geodf['spaces'] + geodf['c_zip_code']

In [46]:
geodf[geodf['g_unit'].notnull()]

,address,address1,address2,c_street_number,c_street,c_unit,c_city,c_state,c_zip_code,g_street_number,g_street,g_unit,g_city,g_state,g_zip_code,g_county,lat,lon,spaces,adj_address
10,11203 RANCH ROAD 2222 APT 1001 AUSTIN TX 78730,11203 RANCH RD 2222 #1001 AUSTIN TX 78730,RANCH RD 2222 #1001 AUSTIN TX 78730,11203,RANCH ROAD 2222,1001,AUSTIN,TX,78730,11203,Ranch Road 2222,1001,Austin,Texas,78730.0,Travis County,30.399156,-97.851646,,11203 RANCH ROAD 2222 AUSTIN TX 78730
20,1036 LIBERTY PARK DR #24 AUSTIN TX 78746,1036 LIBERTY PARK DR #24 AUSTIN TX 78746,LIBERTY PARK DR #24 AUSTIN TX 78746,1036,LIBERTY PARK DRIVE,24,AUSTIN,TX,78746,1036,Liberty Park Drive,24,Austin,Texas,78746.0,Travis County,30.267560,-97.789556,,1036 LIBERTY PARK DRIVE AUSTIN TX 78746
118,1036 LIBERTY PARK DRIVE #24 AUSTIN TX 78746,1036 LIBERTY PARK DR #24 AUSTIN TX 78746,LIBERTY PARK DR #24 AUSTIN TX 78746,1036,LIBERTY PARK DRIVE,24,AUSTIN,TX,78746,1036,Liberty Park Drive,24,Austin,Texas,78746.0,Travis County,30.267560,-97.789556,,1036 LIBERTY PARK DRIVE AUSTIN TX 78746
148,1601 FARO DR UNIT 2501 AUSTIN TX 78741,1601 FARO DR #2501 AUSTIN TX 78741,FARO DR #2501 AUSTIN TX 78741,1601,FARO DRIVE,2501,AUSTIN,TX,78741,1601,Faro Drive,2501,Austin,Texas,78741.0,Travis County,30.233058,-97.711289,,1601 FARO DRIVE AUSTIN TX 78741
340,4500 EAST OLTORF ST #414 AUSTIN FX 78741,4500 EAST OLTORF ST #414 AUSTIN FX 78741,EAST OLTORF ST #414 AUSTIN FX 78741,4500,EAST OLTORF STREET,414,AUSTIN,TX,78741,4500,East Oltorf Street,414,Austin,Texas,78741.0,Travis County,30.226641,-97.724530,,4500 EAST OLTORF STREET AUSTIN TX 78741
366,800 BRAZOS ST UNIT 803 AUSTIN TX 78701,800 BRAZOS ST # 803 AUSTIN TX 78701,BRAZOS ST # 803 AUSTIN TX 78701,800,BRAZOS STREET,803,AUSTIN,TX,78701,800,Brazos Street,803,Austin,Texas,78701.0,Travis County,30.270071,-97.741012,,800 BRAZOS STREET AUSTIN TX 78701
404,1601 FARO DR UNIT 1503 AUSTIN TX 78741,1601 FARO DR #1503 AUSTIN TX 78741,FARO DR #1503 AUSTIN TX 78741,1601,FARO DRIVE,1503,AUSTIN,TX,78741,1601,Faro Drive,1503,Austin,Texas,78741.0,Travis County,30.233058,-97.711289,,1601 FARO DRIVE AUSTIN TX 78741
432,5224 MCCARTY LANE UNIT B AUSTIN TX 78749,5224 MCCARTY LN #B AUSTIN TX 78749,MCCARTY LN #B AUSTIN TX 78749,5224,MCCARTY LANE,B,AUSTIN,TX,78749,5224,McCarty Lane,b,Austin,Texas,78749.0,Travis County,30.226786,-97.851517,,5224 MCCARTY LANE AUSTIN TX 78749
462,54 RAINEY ST - APT 1202 AUSTIN TX 78701,54 RAINEY ST - #1202 AUSTIN TX 78701,RAINEY ST - #1202 AUSTIN TX 78701,54,RAINEY STREET,1202,AUSTIN,TX,78701,54,Rainey Street,1202,Austin,Texas,78701.0,Travis County,30.256902,-97.739764,,54 RAINEY STREET AUSTIN TX 78701
465,909 REINLI STREET APT 112 AUSTIN TX 78751,909 REINLI STREET #112 AUSTIN TX 78751,REINLI STREET #112 AUSTIN TX 78751,909,REINLI STREET,112,AUSTIN,TX,78751,909,Reinli Street,112,Austin,Texas,78751.0,Travis County,30.318825,-97.710421,,909 REINLI STREET AUSTIN TX 78751


In [74]:
import numpy as np
temp = geodf
temp['dummy'] = pd.Series(np.ones(len(temp)))
temp2 = temp.groupby('address').sum()
print(temp2[temp2['dummy'] >5].shape)
print(geodf.shape)


(751, 4)
(518974, 21)


In [40]:
mergedf = newpricedf.merge(geodf, how='left', on='adj_address')

In [41]:
mergedf

,address_x,city_state_zip,price,sqft,price_per_sqft,adj_address,apts,address_y,address1,address2,c_street_number,c_street,c_unit,c_city,c_state,c_zip_code,g_street_number,g_street,g_unit,g_city,g_state,g_zip_code,g_county,lat,lon,spaces
0,4159 STECK AVE #102,AUSTIN TX 78759,280000,1427,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102,4159 STECK AVE APT 195 AUSTIN TX 78759,NaN,NaN,4159,STECK AVENUE,195,AUSTIN,TX,78759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,4159 STECK AVE #102,AUSTIN TX 78759,280000,1427,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102,4159 STECK AVENUE UNIT 235 AUSTIN TX 78759,4159 STECK AVE #235 AUSTIN TX 78759,STECK AVE #235 AUSTIN TX 78759,4159,STECK AVENUE,235,AUSTIN,TX,78759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,4159 STECK AVE #102,AUSTIN TX 78759,280000,1427,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102,4159 STECK AVENUE UNIT 157 AUSTIN TX 78759-8519,NaN,NaN,4159,STECK AVENUE,157,AUSTIN,TX,78759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,4159 STECK AVE #102,AUSTIN TX 78759,280000,1427,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102,4159 STECK AVENUE 239 AUSTIN TX 78759,NaN,NaN,4159,STECK AVENUE,239,AUSTIN,TX,78759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,4159 STECK AVE #102,AUSTIN TX 78759,280000,1427,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102,4159 STECK AVE UNIT 236 AUSTIN TX 78759,4159 STECK AVE #236 AUSTIN TX 78759,STECK AVE #236 AUSTIN TX 78759,4159,STECK AVENUE,236,AUSTIN,TX,78759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5,4159 STECK AVE #102,AUSTIN TX 78759,280000,1427,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102,4159 STECK AVE #175 AUSTIN TX 78759,4159 STECK AVE #175 AUSTIN TX 78759,STECK AVE #175 AUSTIN TX 78759,4159,STECK AVENUE,175,AUSTIN,TX,78759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
6,4159 STECK AVE #102,AUSTIN TX 78759,280000,1427,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102,4159 STECK AVENUE APT 102 AUSTIN TX 78759,4159 STECK AVE #102 AUSTIN TX 78759,STECK AVE #102 AUSTIN TX 78759,4159,STECK AVENUE,102,AUSTIN,TX,78759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
7,4159 STECK AVE #102,AUSTIN TX 78759,280000,1427,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102,4159 SLEEK AVENUE 238 AUSTIN TEXAS 78759,NaN,NaN,4159,STECK AVENUE,238,AUSTIN,TX,78759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,4159 STECK AVE #102,AUSTIN TX 78759,280000,1427,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102,4159 STECK AVENUE UNIT 194 AUSTIN TX 78759,NaN,NaN,4159,STECK AVENUE,194,AUSTIN,TX,78759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
9,4159 STECK AVE #102,AUSTIN TX 78759,280000,1427,196.215837,4159 STECK AVENUE AUSTIN TX 78759,#102,4159 STECK AVE #148 AUSTIN TX 78759,4159 STECK AVE #148 AUSTIN TX 78759,STECK AVE #148 AUSTIN TX 78759,4159,STECK AVENUE,148,AUSTIN,TX,78759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [78]:
latlongs = pd.read_pickle('src/scraping/trulia/sel_scrape/latlonglist.pkl')

In [79]:
latlongs

[('4159 STECK AVENUE AUSTIN TX 78759', '30.2653153', '-97.7495035'),
 ('15810 DE PEER COVE AUSTIN TX 78717', '30.2653153', '-97.7495035'),
 ('8201 LAZY LANE AUSTIN TX 78757', '30.2653153', '-97.7495035'),
 ('14511 FITZGIBBON DRIVE AUSTIN TX 78725', '30.2653153', '-97.7495035'),
 ('10304 BANKHEAD DRIVE AUSTIN TX 78757', '30.2653153', '-97.7495035'),
 ('8505 GRAYLEDGE DRIVE AUSTIN TX 78753', '30.2653153', '-97.7495035'),
 ('1800 LAVACA STREET AUSTIN TX 78701', '30.2653153', '-97.7495035'),
 ('1718 SUMMIT VIEW AUSTIN TX 78703', '30.2653153', '-97.7495035'),
 ('11825 SHROPSHIRE BOULEVARD AUSTIN TX 78753', '30.2653153', '-97.7495035'),
 ('10200 COPPER RIDGE COVE AUSTIN TX 78747', '30.2653153', '-97.7495035'),
 ('8207 DIXON DRIVE AUSTIN TX 78745', '30.2653153', '-97.7495035'),
 ('541 CATALINA LANE AUSTIN TX 78737', '30.2653153', '-97.7495035'),
 ('166 BRIARPATCH COURT AUSTIN TX 78737', '30.2653153', '-97.7495035'),
 ('1401 E RUNDBERG LANE AUSTIN TX 78753', '30.2653153', '-97.7495035'),
 ('20